In [3]:
import os
import pymysql
import os.path

conn = None
host = os.getenv('#MYSQL_HOST')

def createTables():
    try:
        with conn.cursor() as cursor:
            query = "SELECT * FROM information_schema.tables WHERE table_name = 'customer_table' and table_schema = '%s'"%(db)
            cursor.execute(query)
            y = cursor.rowcount
            if y == 0:
                print("Please wait till tables are created")
                query = ""
                query = """create table branch_table(
                    branch_num int unsigned unique auto_increment,
                    branch_id varchar(15) primary key,
                    branch_name varchar(20) not null
                );"""
                cursor.execute(query)
                query = """create trigger branch_insert 
                    before insert 
                    on branch_table 
                    for each row
                    set new.branch_id=concat("ABC",coalesce((select lpad(convert((max(branch_num)+1),char),5,'0') from branch_table),'00001'));"""
                cursor.execute(query)
                query = """create table customer_table(
                customer_name varchar(20) not null,
                customer_id int unsigned auto_increment primary key,
                customer_phone bigint(10) unsigned not null,
                check (char_length(convert(customer_phone,char)) = 10),
                secret_question varchar(50) not null,
                secret_answer varchar(50) not null
                );"""
                cursor.execute(query)
                query = """create table account_table (
                customer_id int unsigned not null,
                branch_id varchar(15) not null,
                account_id varchar(30) not null unique,
                pin int unsigned not null,
                balance float8 unsigned,
                check (balance >500),
                primary key(customer_id,branch_id),
                foreign key (customer_id) references customer_table(customer_id),
                foreign key (branch_id) references branch_table(branch_id)
                );"""
                cursor.execute(query)
                query = """create table bal_table(
                t_no bigint unsigned auto_increment unique,
                branch_id varchar(15),
                customer_id int unsigned,
                account_id varchar(30) not null,
                amount float8 not null,
                check (amount >= 500 or amount <= -500),
                foreign key (branch_id) references branch_table(branch_id),
                foreign key (customer_id) references customer_table(customer_id),
                foreign key (account_id) references account_table(account_id)
                );"""
                cursor.execute(query)
                query = """create trigger account_insert 
                before insert
                on account_table
                for each row
                set new.account_id = concat(new.branch_id,lpad(convert((new.customer_id),char),5,'0'));"""
                cursor.execute(query)
                query = """create view branch_view as select * from branch_table;"""
                cursor.execute(query)
                query = """create view customer_view as select * from customer_table;"""
                cursor.execute(query)
                query = """create view account_view as select * from account_table;"""
                cursor.execute(query)
                query = """create view bal_view as select * from bal_table"""
                cursor.execute(query)
                query = """insert into branch_table(branch_name) values('erode');"""
                cursor.execute(query)
                query = """insert into branch_table(branch_name) values('chennai');"""
                cursor.execute(query)
                query = """insert into branch_table(branch_name) values('coimbatore');"""
                cursor.execute(query)
                query = """insert into branch_table(branch_name) values('tripur');"""
                cursor.execute(query)
                query = """insert into branch_table(branch_name) values('salem');"""
                cursor.execute(query)
                query = """insert into branch_table(branch_name) values('namakkal');"""
                cursor.execute(query)
                query = """create procedure insert_account(customerid int,branchid varchar(15),bal float8,pin int)
                begin
                    declare p char(4);
                    set p = convert(pin,char);
                    if bal <= 500 then
                        signal sqlstate '45000'
                            set message_text = 'Please ensure that you deposit minimum balance';
                    else
                        if length(p) = 4 then
                            insert into account_table(customer_id, branch_id,BALANCE,pin) values(customerId,branchid,bal,pin);
                            select account_id from account_table where customer_id = customerid and branch_id = branchid;
                        else
                        signal sqlstate '45000'
                            set message_text = 'Please enter a valid pin';
                        end if;
                    end if;
                end;"""
                cursor.execute(query)
                query = """create procedure deposit(customer_id int,branch_id varchar(15),account_id varchar(30), amount float8 )
                begin
                if amount > 500 then
                    insert into bal_table(branch_id, customer_id,account_id,amount) values(branch_id,customer_id,account_id,amount);
                    update account_table set balance = balance + amount WHERE account_id = account_id;
                else
                    signal sqlstate '45000'
                        set message_text = 'The minimum deposit amount in 500';
                end if;
                end;"""
                cursor.execute(query)
                query = """create procedure withdraw(customer_id int,branch_id varchar(15), account_id varchar(30), amount float8)
                begin
                declare bal float8;
                select balance from account_table where account_id  = account_id;
                if bal - amount <500 then
                    signal sqlstate '45000'
                        set message_text = 'Minimun balance is not sufficient to make the withdrawl';
                else
                    set amount = amount * -1;
                    insert into bal_table(branch_id, customer_id,account_id,amount) values(branch_id,customer_id,account_id,amount);
                    update account_table set balance = balance + amount WHERE account_id = account_id;
                end if;
                end;"""
                cursor.execute(query)
                print("We are good to go")
            else:
                print("We are good to go")
                pass
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("---------------------------------")
    finally:
        conn.commit()
x = True
flag = True
if os.path.isfile('abc-bank-db.txt'):
    c = int(input("Change database used --->1 \n continue ---> 2"))
    if c==1:
        flag = True
    else:
        flag = False
else:
    flag = True

while x:
    if flag == True:
        usr = input("Enter the username to the db connection")
        password = input("Enter the password of the user")
        db = input("Enter the database you want to use")
        
        try:
            fp = open('abc-bank-db.txt','w')
            lines = [usr+str(','),password+str(','),db]
            fp.writelines(lines)
        except Exception as e:
            print("--------Exception occured--------")
            print(e)
            print("---------------------------------")
    else:
        try:
            fp = open('abc-bank.txt','r')
            res = fp.readlines()
            usr,password,db = res[0].split(',')
        except Exception as e:
            print("--------Exception occured--------")
            print(e)
            print("---------------------------------")

    try:
        conn = pymysql.connect(
        host = host,
        port = int(3306),
        user = usr,
        passwd = password,
        database = db,
        charset = "utf8mb4"
        )
        x = False
        print("Established the connection")
    except Exception as e:
        print("Please enter a valid database name")
        print(e)

createTables()

def dispBranches():
    try:
        with conn.cursor() as cursor:
            query = "select branch_id,branch_name from branch_view;"
            cursor.execute(query)
            for i in cursor:
                print(i[1]," -------> ",i[0])
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("---------------------------------")

def create():
    try:
        with conn.cursor() as cursor:
            choice = int(input("New customer ---> 1\nAlready have an account ---> 2\nExit ---> 3\n"))
            if choice == 1:
                name = input("Enter your name")
                num = int(input("Enter your Indian phone number(Note: It must be unique, no other account holders can have this number)"))
                secq = input("Enter a secret question in order to retreive you account in case of any emergencies (Eg: Who is your favourite hero? )")
                seca = input("Enter the answer for the secret question in order to retreive your account in case of any emergencies ( Eg: My name is akash )")
                query = "insert into customer_table(customer_name,customer_phone,secret_question,secret_answer) values('%s',%s,'%s','%s')"%(name,num,secq,seca)
                cursor.execute(query)
                customer_id = cursor.lastrowid
                print("\nYour new CUSTOMER ID:",customer_id,".You have to remember this id for further details\n")
            elif choice == 2:
                customer_id = input("Enter your customer id")
            elif choice == 3:
                return False,0,''
            else:
                print("Please enter a valid choice!!!")
                return False,0,''
            dispBranches()
            bran = input("Enter the ID of the corresponding branch in which you want to open an account")
            pin = int(input("Enter your 4-DIGIT password for this account"))
            bal = float(input("Enter how much you want to deposit(>500)"))
            para = [customer_id,bran,bal,pin]
            cursor.callproc("insert_account",para)
            account_id = (cursor.fetchall()[0])[0]
            print("Your new ACCOUNT NUMBER :",account_id)
            print("Account has been created successfully !!!!")
            return True,customer_id,account_id
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("---------------------------------")
        return False,0,''
    finally:
        conn.commit()
    
def login():
    try:
        with conn.cursor() as cursor:
            account_id = input("Enter the account id")
            pin = int(input("Enter the 4-DIGIT PIN"))
            query = "select customer_id,branch_id,account_id from account_view where account_id = '%s' and pin = %s;"%(account_id,pin)
            cursor.execute(query)
            if cursor.rowcount == 1:
                res = cursor.fetchone()
                return True,res[0],res[2]
                print(res)
            else:
                raise Exception("Invalid credentials. Please try again!!!")
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("Please check your details and try again!!!")
        print("---------------------------------")
        return False,0,''

def dispChoices(customerid,accountid):
    print("Account Number : ",accountid)
    print("Customer Number : ",customerid)
    with conn.cursor() as cursor:
        try:
            query = "select balance from account_view where account_id = '%s'"%accountid
            cursor.execute(query)
            bal = cursor.fetchone()[0]
            
        except Exception as e:
            print("--------Exception occured--------")
            print(e)            
            print("---------------------------------")
    print("Available balance :",round(bal,2))
    print("Deposit -----> 1")
    print("Withdraw -----> 2")
    print("Get Statement ---> 3")
    print("Log out -----> 4")
    choice = int(input())
    return choice

def deposit(accountid,customerid):
    amount = float(input("Enter the amount you want to deposit(>500)"))
    try:
        arg = [customerid,accountid[0:8],accountid,amount]
        with conn.cursor() as cursor:
            cursor.callproc('deposit',arg)
            print("Amount successfully deposited....!")
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("Please try again!!!")
        print("---------------------------------")
    finally:
        conn.commit()

def withdraw(accountid,customerid):
    amount = float(input("Enter the amount you want to withdraw(>500)"))
    try:
        arg = [customerid,accountid[0:8],accountid,amount]
        with conn.cursor() as cursor:
            cursor.callproc('withdraw',arg)
            print("Amount successfully wtihdrawn....!")
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("Please try again!!!")
        print("---------------------------------")
    finally:
        conn.commit()

def getstatement(customerid,accountid):
    print("Account Statement")
    try:
        with conn.cursor() as cursor:
            query = "select amount from bal_view where account_id = account_id ORDER BY T_NO DESC; "
            cursor.execute(query)
            if cursor.rowcount == 0:
                print("You haven't made any transaction yet")
                return
            print("----------------------------------")
            for i in cursor:
                typ = ''
                if i[0] < 0:
                    typ = "Withdrawal"
                else:
                    typ = "Deposit"
                print("Type :",typ)
                print("Amount :",i[0] * (-1 if i[0]<0 else 1))
                print("----------------------------------")
    except Exception as e:
        print("--------Exception occured--------")
        print(e)
        print("---------------------------------")
        

def forgot(ch):
    if ch == 1:
        num = int(input("Enter your mobile number that is associated with your customer id( like: 1234567890 )"))
        try:
            with conn.cursor() as cursor:
                query = "select account_id from account_view where customer_id = (select customer_id from customer_view where customer_phone = %s )"%num
                cursor.execute(query)
                if cursor.rowcount == 0:
                    print("There are no accounts with the following mobile number")
                    return
                print("Accounts mapped with your phone number:")
                for i in cursor:
                    print(i[0])
        except Exception as e:
            print("--------Exception occured--------")
            print(e)
            print("---------------------------------")
    elif ch == 2:
        num = int(input("Enter your mobile number that is associated with your customer id( like: 1234567890 )"))
        try:
            with conn.cursor() as cursor:
                query = "select customer_id from customer_view where customer_phone = %s"%(num)
                cursor.execute(query)
                if cursor.rowcount == 0:
                    print("There are no accounts with the following mobile number")
                    return
                print("Your customer id is: ")
                for i in cursor:
                    print(i[0])
        except Exception as e:
            print("--------Exception occured--------")
            print(e)
            print("---------------------------------")
    elif ch == 3:
        try:
            acc = input("Enter your account id whose password you forgot")
            num = int(input("Enter your registered phone number"))
            if acc == '':
                raise Exception("Please enter valid account details")
                return
            with conn.cursor() as cursor:
                query = "select * from account_view where account_id = '%s'"%(acc)
                cursor.execute(query)
                if cursor.rowcount == 0:
                    raise Exception("Invalid credentials please try again")
                    return
                query = "select secret_question from customer_view where customer_phone = %s and customer_id = (select customer_id from account_view where account_id = '%s')"%(num,acc)
                cursor.execute(query)
                if cursor.rowcount == 0:
                    raise Exception("Invalid credentials please try again")
                print("Answer your secret question :")
                que = ''
                for i in cursor:
                    print(i[0])
                    que = i[0]
                seca = input("Enter the answer to your secret question")
                query = "select account_id from account_view where customer_id = (select customer_id from customer_view where customer_phone = %s and secret_question= '%s' and secret_answer = '%s');"%(num,que,seca)
                cursor.execute(query)
                if cursor.rowcount == 0:
                    raise Exception("Invalid credentials please try again...!!!")
                    return
                pin = int(input("Enter your new 4 DIGIT-PIN"))
                query = "update account_table set pin = %s where account_id = '%s'"%(pin,acc)
                cursor.execute(query)
                print("PIN changed successfully...!")
        except Exception as e:
            print("--------Exception occured--------")
            print(e)
            print("---------------------------------")
        finally:
            conn.commit()
    elif ch == 4:
        return
    else:
        print("Please enter a valid choice...!")

x = True
loggedIn = False
while x:
    try:
        choice = int(input("Create an account ---> 1\nLogin ---> 2\nForgot Credentials ---> 3\nBANK POLICIES ---> 4\nExit ---> 5\n"))
        print("\n\n")
    except:
        print("Please enter a valid choice")
    if choice == 1:
        loggedIn,customer_id,account_id = create()
        while loggedIn:
            ch = dispChoices(accountid = account_id,customerid=customer_id)
            if ch == 1:
                deposit(accountid = account_id,customerid=customer_id)
            elif ch == 2:
                withdraw(accountid = account_id,customerid=customer_id)
            elif ch ==3:
                getstatement(accountid = account_id,customerid=customer_id)
            elif ch == 4:
                loggedIn = False
            else:
                print("Enter a valid choice")
    elif choice == 2:
        loggedIn,customer_id,account_id = login()
        while loggedIn:
            ch = dispChoices(accountid = account_id,customerid=customer_id)
            if ch == 1:
                deposit(accountid = account_id,customerid=customer_id)
            elif ch == 2:
                withdraw(accountid = account_id,customerid=customer_id)
            elif ch == 3:
                getstatement(accountid = account_id,customerid=customer_id)
            elif ch == 4:
                loggedIn = False
            else:
                print("Enter a valid choice")
    elif choice == 3:
        ch = int(input("Forgot account id ---> 1\nForgot customer id --->2\nForgot PIN --->3\nExit--->4\n"))
        forgot(ch)
    elif choice == 4:
        print("""ABC BANK POLICIES:

1) A customer can have only one customer id.

2) For a single customer id a customer can create only one account per branch.

3) A customer must maintain a minimum balance of 500 and above in his/her account.

4) While creating account a customer must use his/her own phone number and it must not be used for other accounts.

5) A customer must also remember the answer for his secret question since it cannot be changed.

6) While withdrawl or deposit a customer must withdraw or deposit atleast more than 500 in order for the transaction to proceed.""")
    elif choice == 5:
        x = False
    else:
        print("Please enter a valid choice")
    print('\n\n')
    
conn.commit()
conn.close()

Enter the username to the db connectionroot
Enter the password of the userAbcd@1234
Enter the database you want to usetest
Established the connection
Please wait till tables are created
We are good to go
Create an account ---> 1
Login ---> 2
Forgot Credentials ---> 3
BANK POLICIES ---> 4
Exit ---> 5
1



New customer ---> 1
Already have an account ---> 2
Exit ---> 3
1
Enter your nameakash
Enter your Indian phone number(Note: It must be unique, no other account holders can have this number)9976607000
Enter a secret question in order to retreive you account in case of any emergencies (Eg: Who is your favourite hero? )when is your birthday?
Enter the answer for the secret question in order to retreive your account in case of any emergencies ( Eg: My name is akash )july

Your new CUSTOMER ID: 1 .You have to remember this id for further details

erode  ------->  ABC00001
chennai  ------->  ABC00002
coimbatore  ------->  ABC00003
tripur  ------->  ABC00004
salem  ------->  ABC00005
namakkal 